In [1]:
from sklearn.datasets import make_classification
import numpy as np


In [2]:
import xgboost as xgb

In [3]:
# make a synthetic ranking dataset for demonstraction
seed=1994
X,y=make_classification(random_state=seed)
rng=np.random.default_rng(seed)
n_query_groups=3
qid=rng.integers(0,n_query_groups,size=X.shape[0])

In [4]:
# sort the inputs based on query index
sorted_idx=np.argsort(qid)
X=X[sorted_idx]
y=y[sorted_idx]
qid=qid[sorted_idx]

In [6]:
ranker=xgb.XGBRanker(tree_method="hist",lambdarank_num_pair_per_sample=8,objective="rank:ndcg",lambdarank_pair_method="topk")
ranker.fit(X,y,qid=qid) # remove the extra indexing with sorted_idx

XGBRanker(base_score=None, booster=None, callbacks=None, colsample_bylevel=None,
          colsample_bynode=None, colsample_bytree=None, device=None,
          early_stopping_rounds=None, enable_categorical=False,
          eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
          importance_type=None, interaction_constraints=None,
          lambdarank_num_pair_per_sample=8, lambdarank_pair_method='topk',
          learning_rate=None, max_bin=None, max_cat_threshold=None,
          max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
          max_leaves=None, min_child_weight=None, missing=nan,
          monotone_constraints=None, multi_strategy=None, n_estimators=None,
          n_jobs=None, ...)

In [7]:
import pandas as pd


In [8]:
# `X`, `qid`, and `y` are from the previous snippet, they are all sorted by the `sorted_idx`.
df=pd.DataFrame(X,columns=[str(i) for i in range(X.shape[1])])
df["qid"]=qid

In [9]:
ranker.fit(df,y)# no need to pass qid as a separate argument

XGBRanker(base_score=None, booster=None, callbacks=None, colsample_bylevel=None,
          colsample_bynode=None, colsample_bytree=None, device=None,
          early_stopping_rounds=None, enable_categorical=False,
          eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
          importance_type=None, interaction_constraints=None,
          lambdarank_num_pair_per_sample=8, lambdarank_pair_method='topk',
          learning_rate=None, max_bin=None, max_cat_threshold=None,
          max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
          max_leaves=None, min_child_weight=None, missing=nan,
          monotone_constraints=None, multi_strategy=None, n_estimators=None,
          n_jobs=None, ...)

In [10]:
from sklearn.model_selection import StratifiedGroupKFold,cross_val_score

In [11]:
# works with cv in scikit-learn ,along with HPO utilities like GridSearchCV
kfold=StratifiedGroupKFold(shuffle=False)
cross_val_score(ranker,df,y,cv=kfold,groups=df.qid)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [20:16:42] WARNING: /workspace/src/common/error_msg.cc:52: Empty dataset at worker: 0
  warnings.warn(smsg, UserWarning)


array([1., 1., 1., 0., 0.])

In [12]:
scores=ranker.predict(X)
sorted_idx=np.argsort(scores)[::-1]
# Sort the relevance scores from most relevant to least relevant
scores=scores[sorted_idx]